In [4]:
%pip install soundfile
%pip install librosa

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
def genera_lista_da_csv(file_csv, nome_output):
    lista = []
    with open(file_csv, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) 
        for riga in reader:
            nome_file_audio = riga[17]  
            uccello = riga[2] 
            lista.append(f"{uccello}/{nome_file_audio}")

    with open(nome_output, 'w') as file_lista:
        for elemento in lista:
            file_lista.write(elemento + '\n')

def read_list(list_file):
    birds = set()
    with open(list_file, 'r') as f:
        for line in f:
            bird = line.split('/')[0]  
            birds.add(bird)
    return list(birds)

def create_birds_array(list_file):
    birds = read_list(list_file)
    return birds

file_csv = 'bird_dataset_2/bird_songs_metadata.csv'  
nome_output = 'list2.txt'  
genera_lista_da_csv(file_csv, nome_output)
CLASSES = create_birds_array('list2.txt')
print(CLASSES)

['polyglottos', 'migratorius', 'bewickii', 'melodia', 'cardinalis']


In [41]:
import csv
from pathlib import Path
import soundfile as sf
import numpy as np
import librosa
import random
import zipfile
from keras.utils import to_categorical
import io
import copy
import os


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation, Dropout
from keras.activations import softmax
from keras.utils import get_file

In [157]:
def get_bird_name(file_name, mapping_file):
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split('/')
            if len(parts) == 2:
                file_id = file_name.split('/')[-1].split('.')[0] 
                audio_id = parts[1].replace('.wav', '') 
                if file_id == audio_id:
                    return parts[0]
    return None

def check_if_in_testing_list(recording, testing_list):
    recording_id = recording.stem.split('/')[-1].split('.')[0]
    for item in testing_list:
        item_id = Path(item).stem.split('/')[-1]
        if recording_id == item_id:
            return True
    return False

def create_testing_list(list_file):
    with open(list_file, "r") as f:
        lines = f.readlines()
    testing_lines = random.sample(lines, 250)
    with open("testing_list2.txt", "w") as f:
        f.writelines(testing_lines)


def repair_dimensions_y(y_train, y_test):
    if y_train.shape[1] < y_test.shape[1]:
        diff = y_test.shape[1] - y_train.shape[1]
        last_element = y_train[:, -1].reshape(-1, 1)
        y_train = np.hstack((y_train, np.tile(last_element, (1, diff))))
    elif y_test.shape[1] < y_train.shape[1]:
        diff = y_train.shape[1] - y_test.shape[1]
        last_element = y_test[:, -1].reshape(-1, 1)
        y_test = np.hstack((y_test, np.tile(last_element, (1, diff))))
    return y_train, y_test


In [158]:
def creates_test_train():
    create_testing_list("list2.txt")
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    dataset_dir = Path("bird_dataset_2/wavfiles")

    for recording in dataset_dir.iterdir():
        if recording.suffix == ".wav":  
            file_name = recording.stem
            bird_name = get_bird_name(file_name, "list2.txt")
            if bird_name is None or bird_name not in CLASSES:
                continue
            label = CLASSES.index(bird_name)

 
            path_audio = str(recording).replace('\\', '/')
            data, sample_rate = sf.read(path_audio)     
            data = (data * (2 ** 15)).astype(np.int16)
            data = data.astype(np.float32)
            data_resized = np.resize(data, (500, 1))

            
            with open("testing_list2.txt", "r") as f:
                testing_list = f.read().splitlines()

            if check_if_in_testing_list(recording, testing_list):
                x_test.append(data_resized)
                y_test.append(label)
            else:
                x_train.append(data_resized)
                y_train.append(label)

    y_train = to_categorical(np.array(y_train))
    y_test = to_categorical(np.array(y_test))

    if y_train.shape[1] != y_test.shape[1]:
        y_train, y_test = repair_dimensions_y(y_train, y_test)

    x_train = np.array(x_train)
    x_test = np.array(x_test)

    print("Size of x_train:", x_train.shape)
    print("Size of y_train:", y_train.shape)
    print("Size of x_test:", x_test.shape)
    print("Size of y_test:", y_test.shape)

    return x_train, x_test, y_train, y_test

In [159]:
x_train, x_test, y_train, y_test = creates_test_train()

Size of x_train: (1774, 500, 1)
Size of y_train: (1774, 5)
Size of x_test: (89, 500, 1)
Size of y_test: (89, 5)


In [160]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

np.savetxt('x_test.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test.csv', y_test, delimiter=',', fmt='%s')

In [161]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_normalized = scaler.transform(x_test.reshape(x_test.shape[0], -1))


x_train = x_train_normalized.reshape(x_train.shape)
x_test = x_test_normalized.reshape(x_test.shape)


Now there is the choice of the model, run the one desired

This is the model based on the paper 

In [162]:
model = Sequential()
model.add(Input(shape=( 500, 1)))

model.add(Conv1D(filters=16, kernel_size=80))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=4, strides=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1))

model.add(AvgPool1D(pool_size=44))

model.add(Flatten())
#model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=len(CLASSES)))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_108 (Conv1D)         (None, 421, 16)           1296      
                                                                 
 activation_74 (Activation)  (None, 421, 16)           0         
                                                                 
 max_pooling1d_62 (MaxPooli  (None, 209, 16)           0         
 ng1D)                                                           
                                                                 
 conv1d_109 (Conv1D)         (None, 207, 32)           1568      
                                                                 
 max_pooling1d_63 (MaxPooli  (None, 102, 32)           0         
 ng1D)                                                           
                                                                 
 conv1d_110 (Conv1D)         (None, 100, 64)         

This is the ResNet model

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, AveragePooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

input_shape = (500, 1)
num_classes = 5

model = Sequential()


model.add(Conv1D(filters=16, kernel_size=7, strides=2, input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2, strides=2))

for _ in range(10):
    model.add(Conv1D(filters=16, kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    
    model.add(Conv1D(filters=16, kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))



model.add(Conv1D(filters=32, kernel_size=3, strides=2))
model.add(BatchNormalization())
model.add(Activation('relu'))


for _ in range(1):
    model.add(Conv1D(filters=32, kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv1D(filters=32, kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))


model.add(AvgPool1D(pool_size=2))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=num_classes))
model.add(Activation('softmax'))


opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [163]:
model.fit(x_train, y_train, epochs=10, batch_size=300, validation_data=(x_test, y_test))

Epoch 1/10
6/6 [==============================] - 2s 146ms/step - loss: 1.5918 - accuracy: 0.2830 - val_loss: 1.5579 - val_accuracy: 0.2921
Epoch 2/10
6/6 [==============================] - 1s 113ms/step - loss: 1.5414 - accuracy: 0.3298 - val_loss: 1.4875 - val_accuracy: 0.3596
Epoch 3/10
6/6 [==============================] - 1s 115ms/step - loss: 1.4770 - accuracy: 0.3743 - val_loss: 1.4028 - val_accuracy: 0.3708
Epoch 4/10
6/6 [==============================] - 1s 120ms/step - loss: 1.4283 - accuracy: 0.3873 - val_loss: 1.3847 - val_accuracy: 0.3933
Epoch 5/10
6/6 [==============================] - 1s 130ms/step - loss: 1.4070 - accuracy: 0.3918 - val_loss: 1.3667 - val_accuracy: 0.3933
Epoch 6/10
6/6 [==============================] - 1s 123ms/step - loss: 1.3853 - accuracy: 0.3980 - val_loss: 1.3455 - val_accuracy: 0.3820
Epoch 7/10
6/6 [==============================] - 1s 119ms/step - loss: 1.3653 - accuracy: 0.4087 - val_loss: 1.3366 - val_accuracy: 0.4270
Epoch 8/10
6/6 [====

In [164]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

3/3 - 0s - loss: 1.3265 - accuracy: 0.4157 - 32ms/epoch - 11ms/step
3/3 [==============================] - 0s 6ms/step
tf.Tensor(
[[10  0  0 13  1]
 [ 9  2  0  7  0]
 [ 0  0  0  8  1]
 [ 1  0  0 22  3]
 [ 4  0  0  5  3]], shape=(5, 5), dtype=int32)


In [165]:
model.save('model_5spec.h5')

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [166]:
if isinstance(model.layers[-1], Activation) and model.layers[-1].activation == softmax:
    model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)
else:
    print('Error: last layer is not SoftMax Activation')

In [167]:
%pip install qualia_codegen_core
import qualia_codegen_core
from qualia_codegen_core.graph.KerasModelGraph import KerasModelGraph
from qualia_codegen_core.graph.Quantization import Quantization
from qualia_codegen_core.graph.RoundMode import RoundMode

from importlib.resources import files
main_path = str((files('qualia_codegen_core.examples')/'Linux'/'main.cpp').resolve())

Note: you may need to restart the kernel to use updated packages.


In [168]:
modelgraph = KerasModelGraph(model).convert()
print(modelgraph)

—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                                           | Layer                                            | Outputs                                          | Input shape                                      | Output shape                                    
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                                                 | input_13                                         | conv1d_108                                       | (1, 500, 1)                                      | ((1, 500, 1),)                     

In [169]:
float_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for float32
for node in float_modelgraph.nodes:
    # No scale factor if not fixed-point quantization on integers
    node.q = Quantization(
            number_type=float,
            width=32,
            long_width=32,
            weights_scale_factor=0,
            output_scale_factor=0,
            weights_round_mode=RoundMode.NONE,
            output_round_mode=RoundMode.NONE,
            )

float_res = qualia_codegen_core.Converter(output_path=Path('output_floating')).convert_model(float_modelgraph)

with open('model_floating.h', 'w') as f:
    f.write(float_res)

Graphviz not available


In [170]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o floating -include output_floating/include/defines.h -Ioutput_floating/include output_floating/model.c {main_path}
!./floating x_test.csv y_test.csv

In file included from output_floating/model.c:15:
output_floating/include/number.h: In function ‘float scale_number_t_float(float, int, round_mode_t)’:
output_floating/include/number.h:73:21: warning: unused parameter ‘scale_factor’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
   73 |   float number, int scale_factor, round_mode_t round_mode) {
      |                 ~~~~^~~~~~~~~~~~
output_floating/include/number.h:73:48: warning: unused parameter ‘round_mode’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
   73 |   float number, int scale_factor, round_mode_t round_mode) {
      |                                   ~~~~~~~~~~~~~^~~~~~~~~~
output_floating/include/number.h: In function ‘float scale_and_clamp_to_number_t_float(float, int, round_mode_t)’:
output_floating/include/number.h:81:21: warning: unused parameter ‘scale_factor’ []8;;https://gcc.gnu.o

In [171]:
fixed_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for int16 Q9.7
for node in fixed_modelgraph.nodes:
    node.q = Quantization(
            number_type=int,
            width=16,
            long_width=32,
            weights_scale_factor=7,
            output_scale_factor=7,
            weights_round_mode=RoundMode.FLOOR,
            output_round_mode=RoundMode.FLOOR,
            )

fixed_res = qualia_codegen_core.Converter(output_path=Path('output_fixed')).convert_model(fixed_modelgraph)

with open('model_fixed.h', 'w') as f:
    f.write(fixed_res)

Graphviz not available


In [172]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o fixed -include output_fixed/include/defines.h -Ioutput_fixed/include output_fixed/model.c {main_path}
!./fixed x_test.csv y_test.csv

Testing accuracy: 0.449438
